In [4]:
# OPENCV
import cv2 as cv
import numpy as np
from PIL import ImageGrab
from functools import partial
import pyautogui

ImageGrab.grab = partial(ImageGrab.grab, all_screens=True)

In [5]:
def hsvColorSlider():
    cv.namedWindow('HSV Color Slider')
    cv.createTrackbar('Hue Min', 'HSV Color Slider', 0, 179, nothing)
    cv.createTrackbar('Hue Max', 'HSV Color Slider', 104, 179, nothing)
    cv.createTrackbar('Sat Min', 'HSV Color Slider', 0, 255, nothing)
    cv.createTrackbar('Sat Max', 'HSV Color Slider', 255, 255, nothing)
    cv.createTrackbar('Val Min', 'HSV Color Slider', 124, 255, nothing)
    cv.createTrackbar('Val Max', 'HSV Color Slider', 255, 255, nothing)

def nothing(x):
    pass

def getHSV():
    hue_min = cv.getTrackbarPos('Hue Min', 'HSV Color Slider')
    hue_max = cv.getTrackbarPos('Hue Max', 'HSV Color Slider')
    sat_min = cv.getTrackbarPos('Sat Min', 'HSV Color Slider')
    sat_max = cv.getTrackbarPos('Sat Max', 'HSV Color Slider')
    val_min = cv.getTrackbarPos('Val Min', 'HSV Color Slider')
    val_max = cv.getTrackbarPos('Val Max', 'HSV Color Slider')
    return hue_min, hue_max, sat_min, sat_max, val_min, val_max

In [6]:
def convert_to_binary(image):
    hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    # remove background
    (hmin, hmax, smin, smax, vmin, vmax) = getHSV()
    lower = np.array([hmin, smin, vmin])
    upper = np.array([hmax, smax, vmax])
    mask = cv.inRange(hsv, lower, upper)

    krn = cv.getStructuringElement(cv.MORPH_RECT, (50, 30))
    dlt = cv.dilate(mask, krn, iterations=5)
    res = 255 - cv.bitwise_and(dlt, mask)
    # # Apply threshold
    # _, res = cv.threshold(hsv, 0, 255, cv.THRESH_OTSU)
    return res

In [7]:
# detect checkers board
def detect_checkers_board(img):
    # convert to gray scale
    mask = convert_to_binary(img)
    # mask = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # find the chess board corners
    ret, corners = cv.findChessboardCorners(mask, (7, 7), None)
    # if found, draw corners
    if ret == True:
        # draw corners
        cv.drawChessboardCorners(img, (7, 7), corners, ret)
        return True, corners
    return False, None

In [8]:
# get the board corners
def get_board_corners(corners):
    # get the corners
    corners = corners.reshape(-1, 2)
    x_delta = abs(corners[0][0] - corners[1][0])
    y_delta = abs(corners[0][1] - corners[7][1])

    # get the top left corner
    top_left = corners[0][0] - x_delta, corners[0][1] - y_delta
    # get the top right corner
    top_right = corners[6][0] + x_delta, corners[6][1] - y_delta
    # get the bottom left corner
    bottom_left = corners[42][0] - x_delta, corners[42][1] + y_delta
    # get the bottom right corner
    bottom_right = corners[48][0] + x_delta, corners[48][1] + y_delta
    return top_left, top_right, bottom_left, bottom_right

In [9]:
def get_board_squares(squareCoords, corners):
    # get the corners
    corners = corners.reshape(-1, 2)
    x_delta = abs(corners[0][0] - corners[1][0])
    y_delta = abs(corners[0][1] - corners[7][1])
    
    home_coords = corners[0][0] - x_delta/2, corners[0][1] - y_delta/2

    x = home_coords[0] + ((x_delta) * squareCoords[1])
    y = home_coords[1] + ((y_delta) * squareCoords[0])
    coords = (x, y)

    return coords

In [10]:
def crop_board(img, corners):
    # get the corners
    top_left, top_right, bottom_left, bottom_right = get_board_corners(corners)
    # get the width of the board
    width = int(np.sqrt((top_right[0] - top_left[0])**2 + (top_right[1] - top_left[1])**2))
    # get the height of the board
    height = int(np.sqrt((top_left[0] - bottom_left[0])**2 + (top_left[1] - bottom_left[1])**2))
    # get the perspective transform matrix
    pts1 = np.float32([top_left, top_right, bottom_left, bottom_right])
    pts2 = np.float32([[0, 0], [width, 0], [0, height], [width, height]])
    matrix = cv.getPerspectiveTransform(pts1, pts2)
    # apply the perspective transform
    result = cv.warpPerspective(img, matrix, (width, height))
    return result

In [11]:
def detect_pieces(boardimg, corners):
    if corners is not None:
        # get the corners
        top_left, _, _, bottom_right = get_board_corners(corners)
        # detect white squares
        # convert to gray scale
        gray = cv.cvtColor(boardimg, cv.COLOR_BGR2GRAY)
        # apply threshold
        _, thresh = cv.threshold(gray, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)
        # find contours
        contours, _ = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        # get the squares
        # apply only the squares that are inside the board
        pieces = []
        for cnt in contours:
            x, y, w, h = cv.boundingRect(cnt)
            if bottom_right[0] > x > top_left[0] and bottom_right[1] > y > top_left[1]:
                if 25 < w < 50 and 25 < h < 50:
                    pieces.append((x, y, w, h))
    return pieces

In [12]:
def match_images(img, template):
    # convert to gray scale
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    template = cv.cvtColor(template, cv.COLOR_BGR2GRAY)
    # get the template size
    w, h = template.shape[::-1]
    # match the template
    res = cv.matchTemplate(gray, template, cv.TM_CCOEFF_NORMED)
    # get the threshold
    threshold = 0.8
    loc = np.where(res >= threshold), w, h

    return loc

In [13]:
def update_board_state(boardimg):
    pieces = detect_pieces(boardimg)
    board = np.zeros((8, 8), dtype=int)
    for piece in pieces:
        x, y, w, h = piece
        square = (int(x/50), int(y/50))
        board[square] = 1
    return board

In [14]:
# only screen 1
screen = ImageGrab.grab(all_screens=True)
print(f"Resolution: {screen.size}, Mode: {screen.mode}, Format: {screen.format}")

# screen right : (1920, 0, 3840, 1080)
# screen left : (0, 0, 1920, 1080)

screen = screen.crop((1920, 0, 3840, 1080))
screen = np.array(screen)

Resolution: (3840, 1080), Mode: RGB, Format: None


In [20]:
pickerwindowBG = np.zeros((300, 512, 3), np.uint8)
hsvColorSlider()
(hmin, hmax, smin, smax, vmin, vmax) = getHSV()
cv.imshow('HSV Color Slider', pickerwindowBG)
while True:
    screen = ImageGrab.grab(all_screens=True)
    print(f"Resolution: {screen.size}, Mode: {screen.mode}, Format: {screen.format}")

    # screen right : (1920, 0, 3840, 1080)
    # screen left : (0, 0, 1920, 1080)

    screen = screen.crop((1920, 0, 3840, 1080))
    screen = np.array(screen)

    screen_binary = convert_to_binary(screen)
    ret, corners = detect_checkers_board(screen)
    screen = cv.cvtColor(screen, cv.COLOR_BGR2RGB)

    if ret:
        print("Checkers board detected")
        screen_binary = cv.cvtColor(screen_binary, cv.COLOR_GRAY2RGB)

        # get the pieces
        pieces = detect_pieces(screen_binary, corners)
        print(len(pieces), pieces)
        for piece in pieces:
            x, y, w, h = piece
            cv.rectangle(screen_binary, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        screen_binary = crop_board(screen_binary, corners)
        
        cv.imshow('screen_binary', screen_binary)
    else:
        print("Checkers board not detected")
        # check game end
        endimg = cv.imread('end.png')
        matchinfo = match_images(screen, endimg)
        loc, w, h = matchinfo
        if loc[0].size > 0:
            print("Game ended")
            for pt in zip(*loc[::-1]):
                cv.rectangle(screen, pt, (pt[0] + w, pt[1] + h), (0, 0, 255), 2)
        else:
            print("Game not ended")
    # show screen
    cv.imshow('screen', screen)
    # cv.imshow('screen', screen)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
cv.destroyAllWindows()

Resolution: (3840, 1080), Mode: RGB, Format: None
Checkers board detected
16 [(1083, 734, 34, 34), (955, 734, 37, 33), (830, 734, 34, 34), (702, 734, 37, 33), (1018, 673, 36, 33), (765, 673, 36, 33), (892, 671, 35, 33), (1082, 608, 35, 32), (889, 416, 42, 41), (1079, 352, 42, 42), (953, 352, 41, 42), (700, 352, 41, 42), (1142, 289, 42, 42), (1016, 289, 42, 42), (889, 289, 42, 42), (763, 289, 42, 42)]
Resolution: (3840, 1080), Mode: RGB, Format: None
Checkers board detected
16 [(1083, 734, 34, 34), (955, 734, 37, 33), (830, 734, 34, 34), (702, 734, 37, 33), (1018, 673, 36, 33), (765, 673, 36, 33), (892, 671, 35, 33), (1082, 608, 35, 32), (889, 416, 42, 41), (1079, 352, 42, 42), (953, 352, 41, 42), (700, 352, 41, 42), (1142, 289, 42, 42), (1016, 289, 42, 42), (889, 289, 42, 42), (763, 289, 42, 42)]
Resolution: (3840, 1080), Mode: RGB, Format: None
Checkers board detected
16 [(1083, 734, 34, 34), (955, 734, 37, 33), (830, 734, 34, 34), (702, 734, 37, 33), (1018, 673, 36, 33), (765, 673, 3

In [9]:
# get the board corners
if corners is not None:
    top_left, top_right, bottom_left, bottom_right = get_board_corners(corners)
    print(f"Top left: {top_left}, Top right: {top_right}, Bottom left: {bottom_left}, Bottom right: {bottom_right}")
    coords = get_board_squares((1, 0), corners)
    print(coords)

Top left: [896.5236 342.6222], Top right: [1275.5586   343.11987], Bottom left: [896.5 722.5], Bottom right: [1276.0753  722.4799]
(864.7854309082031, 374.06109619140625)


In [10]:
if corners is not None:
    for i in range(8):
        for j in range(8):
            coords = get_board_squares((i, j), corners)
            print(f"Square {i}, {j}: {coords}")
            pyautogui.moveTo(coords)

Square 0, 0: (864.7854309082031, 311.18328857421875)
Square 0, 1: (928.2618103027344, 311.18328857421875)
Square 0, 2: (991.7381896972656, 311.18328857421875)
Square 0, 3: (1055.2145690917969, 311.18328857421875)
Square 0, 4: (1118.6909484863281, 311.18328857421875)
Square 0, 5: (1182.1673278808594, 311.18328857421875)
Square 0, 6: (1245.6437072753906, 311.18328857421875)
Square 0, 7: (1309.1200866699219, 311.18328857421875)
Square 1, 0: (864.7854309082031, 374.06109619140625)
Square 1, 1: (928.2618103027344, 374.06109619140625)
Square 1, 2: (991.7381896972656, 374.06109619140625)
Square 1, 3: (1055.2145690917969, 374.06109619140625)
Square 1, 4: (1118.6909484863281, 374.06109619140625)
Square 1, 5: (1182.1673278808594, 374.06109619140625)
Square 1, 6: (1245.6437072753906, 374.06109619140625)
Square 1, 7: (1309.1200866699219, 374.06109619140625)
Square 2, 0: (864.7854309082031, 436.93890380859375)
Square 2, 1: (928.2618103027344, 436.93890380859375)
Square 2, 2: (991.7381896972656, 436

In [21]:
list = pieces.copy()
print(len(list))
list = sorted(list, key=lambda x: x[0])
list = sorted(list, key=lambda x: x[1])
print(list)

16
[(763, 289, 42, 42), (889, 289, 42, 42), (1016, 289, 42, 42), (1142, 289, 42, 42), (700, 352, 41, 42), (953, 352, 41, 42), (1079, 352, 42, 42), (889, 416, 42, 41), (1082, 608, 35, 32), (892, 671, 35, 33), (765, 673, 36, 33), (1018, 673, 36, 33), (702, 734, 37, 33), (830, 734, 34, 34), (955, 734, 37, 33), (1083, 734, 34, 34)]


In [22]:
newcoords = []
for coords in list:
    x, y, w, h = coords
    pyautogui.moveTo(x + w/2, y + h/2)
    newcoords.append((x + w/2, y + h/2))
print(newcoords)

[(784.0, 310.0), (910.0, 310.0), (1037.0, 310.0), (1163.0, 310.0), (720.5, 373.0), (973.5, 373.0), (1100.0, 373.0), (910.0, 436.5), (1099.5, 624.0), (909.5, 687.5), (783.0, 689.5), (1036.0, 689.5), (720.5, 750.5), (847.0, 751.0), (973.5, 750.5), (1100.0, 751.0)]
